In [1]:
library(BSgenome)
library(rtracklayer)
library(qlcMatrix)
library(dplyr)
library(TFBSTools)
library(BSgenome.Hsapiens.UCSC.hg38)

root = '/Genomics/pritykinlab/zzhao/sc-atac-submmit-calling/sc-atac-submmit-calling'
motif_regression_out = "results/motif_regression_output/"
motif_regression_out = paste(root, motif_regression_out, sep="/")

summit.peaks <- rtracklayer::import(paste(motif_regression_out, "SUMMITS-all-atac-summits.bed", sep="/"))
summits.seq <- getSeq(BSgenome.Hsapiens.UCSC.hg38, summit.peaks)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: GenomicRanges

Loading required package: Biostrings

Loading required package: XVector


Attachin

In [16]:
summits.seq

DNAStringSet object of length 226759:
         width seq
     [1]   150 TAACCCTAACCCAACCCTAACCCTAACCCTA...TAACCCTAACCCTAACCCCTAACCCTAACCC
     [2]   150 TAACCCTAACCCTAACAACCCTAACCCTAAC...CCCCTAACCCCTAACCCTAACCCTAACCCTA
     [3]   150 ATGCTAGCGCGCCCAGGGGAGGAGGCGTGGC...GACGCACGCCTACGGGCGGGGTTGGGGGGGG
     [4]   150 TGCAATGCTAATTTGCCAGGAGCTCACATGC...GCCCCTGTCTCTTCCGTGCAGGAGGAGCATG
     [5]   150 CGTTCCCTATCTGTCACCATTTCCTGTCGTC...GCTTCTGGGTGTCCCCGGCATCCACTCTCTC
     ...   ... ...
[226755]   150 GAAAAGGATCATCACAGTGAGAGTTACAAAC...ACTCCCTATTCTGACCAGGAAGTGGTTAGTG
[226756]   150 TTGTATTGATTTTTCAAGAGAGCCAAGAGGG...ACCATCTCCCCTCAGATGGTGGGAGCAGGGA
[226757]   150 AGTTCCCTGACTTAGAAGATTAGTGGTGACA...GCTCACGTCACCAACTGCTGCAGTTATCTCC
[226758]   150 GGTTAGGGTTAGGGTTAGGGGTTAGGGGTTA...TTAGGGTTAGGGTTAGGGGTTAGGGGTTAGG
[226759]   150 AGGGTTAGGGTTAGGGTTAGGGTTAGGGTTA...TAGGGTTAGGGTTAGGGTTAGGGTTAGGGTT

In [17]:
?writeXStringSet

In [18]:
writeXStringSet(summits.seq, 'results/motif_regression_output/DEEP_LEARNING_summit_seq.fasta', append=FALSE, compress=FALSE, compression_level=NA, format="fasta")

In [3]:
generate_pwm <- function(line_n) {
    file = "data/consensus_pwms.meme"
    name_info = strsplit(read.csv(file, skip=line_n-1, nrows = 1, sep=" ",
                                   header = F)$V2, ":")[[1]]
    len = as.numeric(gsub(" nsites.*", "",
                           gsub(".* w= ", "", read.csv(file, skip=line_n,
                                                       nrows = 1, sep="\t", header = F)$V1)))
    mat = as.matrix(read.csv(file,  skip = line_n + 2,
                                                sep = "\t",  nrows = len, header = FALSE) %>%
                                       transmute(A=V1, C=V2, G=V3, T=V4)) %>% t()
    # nearly 1, but not exactly
    corrected_mat <- apply(mat, 1, "/", colSums(mat)) %>% t() 
    PWMatrix(ID=name_info[1], name=paste(name_info[2], name_info[3], sep="_"),
             bg=c(A=0.25, C=0.25, G=0.25, T=0.25), tags=list(),  pseudocounts=numeric(), 
             profileMatrix = corrected_mat)
}

lines_to_start_from = lapply(system('grep -n "MOTIF " data/consensus_pwms.meme',
                                    intern = T), function(ln) {
    as.numeric(strsplit(ln, ":")[[1]][1])}) %>% unlist(use.names = F)

In [4]:
pwm_mat_list <- do.call(PWMatrixList, lapply(lines_to_start_from, generate_pwm))

In [5]:
pwm_mat_list

PWMatrixList of length 693

In [6]:
library(motifmatchr)

In [7]:
generate_motif_hits <- function(pwms, summit_seqs, p_val_thresh, out_bn){
    motif_hits = matchMotifs(pwms, summit_seqs, p.cutoff = p_val_thresh, out = "scores")
    saveRDS(motif_hits, paste("results/motif_regression_output/matchR_out/motif-matches-", out_bn, ".rds", sep=""))
    motif_hits
}

In [8]:
motif_hits <- generate_motif_hits(pwm_mat_list, summits.seq, 5e-05, 'test')

In [9]:
motif_hits

class: SummarizedExperiment 
dim: 226759 693 
metadata(0):
assays(3): motifScores motifMatches motifCounts
rownames: NULL
rowData names(0):
colnames: NULL
colData names(1): name

In [10]:
head(motifScores(motif_hits))

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)



6 x 693 sparse Matrix of class "dgCMatrix"
                                                                        
[1,] . . . . . . . . . . . . . .        .        . . . . . . . . . . . .
[2,] . . . . . . . . . . . . . .        .        . . . . . . . . . . . .
[3,] . . . . . . . . . . . . . .        .        . . . . . . . . . . . .
[4,] . . . . . . . . . . . . . .        2.555824 . . . . . . . . . . . .
[5,] . . . . . . . . . . . . . .        .        . . . . . . . . . . . .
[6,] . . . . . . . . . . . . . 3.188007 .        . . . . . . . . . . . .
                                                                              
[1,]  .       . . .        . . . . . . . . . . . . . . . . . . . . . .       .
[2,]  .       . . .        . . . . . . . . . . . . . . . . . . . . . 3.43243 .
[3,]  .       . . .        . . . . . . . . . . . . . . . . . . . . . .       .
[4,]  .       . . 4.310619 . . . . . . . . . . . . . . . . . . . . . .       .
[5,] 10.39362 . . .        . . . . . . . . . . . . 

In [11]:
# also works
# library(SummarizedExperiment)
# assays(motif_hits)$motifScores

In [12]:
writeMM(motifScores(motif_hits), "results/motif_regression_output/matchR_out/MOTIF_PBMC_summit_motif_matrix.mtx")

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)



NULL

In [13]:
motif_hits

class: SummarizedExperiment 
dim: 150007 693 
metadata(0):
assays(3): motifScores motifMatches motifCounts
rownames: NULL
rowData names(0):
colnames: NULL
colData names(1): name

In [14]:
head(motifMatches(motif_hits))

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)

dimnames(.) <- NULL:  translated to 
dimnames(.) <- list(NULL,NULL)  <==>  unname(.)



6 x 693 sparse Matrix of class "lgCMatrix"
                                                                              
[1,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[2,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[4,] . . . . . . . . . . . . . . | . . . . . . . . . . . . . . . | . . . . . .
[5,] . . . . . . . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . .
[6,] . . . . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . . .
                                                                              
[1,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[2,] . . . . . . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . .
[3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[4,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
[5,] . . 